# Nuclear

Should get:

 * rms data
 * magnetic moments (recommended)
 * quadrupole moments (recommended)
   * Then the "compiled" data, only if not in above
 * Also: existing list
   * update exiting list with new data from above
   * But don't delete items from existing list!
 * Have extra "manual override" file


For moments:
  * Drop anything not nuclear ground state
  * Spin: get spin and parity from string "3/2-" -> 1.5, -1
  * Moments uncertainty: 1.1234(56) -> 1.1234, 0.0056

For radius:
  * Use prelim data, if available

In [34]:
import numpy as np
from matplotlib import pyplot as plt
import requests
import pandas as pd
from io import BytesIO

In [35]:
mu_url = "https://www-nds.iaea.org/nuclearmoments/magn_mom_recomm.csv"
rrms_url = "https://www-nds.iaea.org/radii/charge_radii.csv"
Q_url = "https://www-nds.iaea.org/nuclearmoments/elec_mom_recomm.csv"
# only use this one to fill any gaps - don't update
muQ_url = "https://www-nds.iaea.org/nuclearmoments/nuc_mom_compilation.csv"


mu_recieved = requests.get(mu_url)
rrms_recieved = requests.get(rrms_url)
Q_recieved = requests.get(Q_url)
muQ_recieved = requests.get(muQ_url)

In [116]:
def convert_to_float_or_nan(string):
    try:
        if "/" in string:
            a, b = string.split("/")
            return float(a) / int(b)
        else:
            return float(string)
    except ValueError:
        return np.nan


def parse_rrms_data(content):

    rrms_data = pd.read_csv(BytesIO(content)).values

    # if there is preliminary rms data, use that
    for row in rrms_data:
        if not np.isnan(row[6]):
            row[4] = row[6]

    # delete any with Z<1 (has neutron)
    rrms_data = np.delete(rrms_data, rrms_data[:, 0] < 1, axis=0)

    # delete collumns we don't need
    rrms_data = np.delete(rrms_data, [1, 2, 5, 6, 7], axis=1)

    # make slots for I, pi, mu, Q
    nans = np.empty((len(rrms_data), 4))
    nans[:] = np.nan
    return np.append(rrms_data, nans, axis=1)


def parse_moment_data(content):

    data = pd.read_csv(BytesIO(content))
    data = data.drop(
        ["symbol", "halflife", "method", "description", "nsr", "journal", "indc"],
        axis=1,
    )

    # Remove neutron, and excited states
    data = data[data["z"] != 0]
    data = data[data["energy [keV]"] == "0"]
    data = data.drop(["energy [keV]"], axis=1)

    # make room for parsed data (I, pi, mu/Q)
    data = data.values
    nans = np.empty((len(data), 3))
    nans[:] = np.nan
    data = np.append(data, nans, axis=1)

    for row in data:
        jpi = row[2]
        # Some values which are uncertain are in parenthesis
        jpi = jpi.replace("(", "").replace(")", "")

        pi = -1 if jpi[-1] == "-" else +1

        jpi = jpi.replace("+", "").replace("-", "")
        jpi = jpi.replace("if", "").replace(" ", "")

        I = convert_to_float_or_nan(jpi)

        row[4] = I
        row[5] = pi

        # when the sign is uncertain, in the form "(+)1.234" or "(-)1.234"
        if "(+)" in row[3]:
            row[3] = row[3].split("(+)")[1]
        if "(-)" in row[3]:
            row[3] = "-" + row[3].split("(-)")[1]

        # Get rid of uncertainties
        string = row[3].split("(")[0].replace(" ", "")
        mu = convert_to_float_or_nan(string)
        row[6] = mu

    # delete collumns we don't need
    return np.delete(data, [2, 3], axis=1)

In [117]:
rrms_data = parse_rrms_data(rrms_recieved.content)
mu_data = parse_moment_data(mu_recieved.content)
Q_data = parse_moment_data(Q_recieved.content)

print(rrms_data.shape)
print(mu_data.shape)
print(Q_data.shape)

(956, 7)
(800, 5)
(533, 5)


In [118]:
def search(data, t_z, t_a):
    for index, row in enumerate(data):
        if row[0] == t_z and row[1] == t_a:
            return index
    return -1


print(rrms_data.shape)

# insert/update values from mu_data
for [z, a, I, pi, mu] in mu_data:
    index = search(rrms_data, z, a)
    if index == -1:
        rrms_data = np.append(rrms_data, [[z, a, np.nan, I, pi, mu, np.nan]], axis=0)
    else:
        rrms_data[index][3] = I
        rrms_data[index][4] = pi
        rrms_data[index][5] = mu

print(rrms_data.shape)

# insert/update values from Q_data
for [z, a, I, pi, q] in Q_data:
    index = search(rrms_data, z, a)
    if index == -1:
        rrms_data = np.append(rrms_data, [[z, a, np.nan, I, pi, np.nan, q]], axis=0)
    else:
        rrms_data[index][3] = I
        rrms_data[index][4] = pi
        rrms_data[index][6] = q

# re-convert floats to integers..
for index in range(len(rrms_data)):
    rrms_data[index][0] = int(rrms_data[index][0])
    rrms_data[index][1] = int(rrms_data[index][1])
    if not np.isnan(rrms_data[index][4]):
        rrms_data[index][4] = int(rrms_data[index][4])

print(rrms_data.shape)

# sort by Z, then A
ind = np.lexsort((rrms_data[:, 0], rrms_data[:, 1]))
rrms_data = rrms_data[ind]

print(rrms_data.shape)

(956, 7)
(1262, 7)
(1270, 7)
(1270, 7)


In [119]:
for [z, a, rrms, I, pi, mu, q] in rrms_data:
    if np.isnan(I) and not np.isnan(mu):
        print([z, a, rrms, I, pi, mu, q])

[31, 74, 4.0395, nan, -1, 0.0, 0.55]


In [120]:
for [z, a, rrms, I, pi, mu, q] in rrms_data:
    if z == 87:
        print([z, a, rrms, I, pi, mu, q])

[87, 202, 5.5505, 3.0, 1, 3.9, nan]
[87, 203, 5.556, 4.5, -1, 3.74, -0.47]
[87, 204, nan, 3.0, 1, 3.99, -0.141]
[87, 205, 5.5609, 4.5, -1, 3.8, -0.305]
[87, 206, nan, 3.0, 1, 3.97, -0.354]
[87, 207, 5.5689, 4.5, -1, 3.87, -0.24]
[87, 208, 5.5699, 7.0, 1, 4.71, 0.052]
[87, 209, 5.578, 4.5, -1, 3.93, -0.26]
[87, 210, 5.5803, 6.0, 1, 4.38, 0.21]
[87, 211, 5.5876, 4.5, -1, 3.97, -0.21]
[87, 212, 5.5915, 5.0, 1, 4.59, -0.1]
[87, 213, 5.5986, 4.5, -1, 3.99, -0.138]
[87, 214, nan, 1.0, -1, 0.241, nan]
[87, 219, nan, 2.5, -1, nan, -1.19]
[87, 220, 5.6803, 1.0, 1, -0.66, 0.487]
[87, 221, 5.692, 2.5, -1, 1.57, -1.02]
[87, 222, 5.7018, 2.0, -1, 0.63, 0.51]
[87, 223, 5.7104, 1.5, -1, 1.16, 1.18]
[87, 224, 5.7229, 1.0, -1, 0.4, 0.523]
[87, 225, 5.7288, 1.5, -1, 1.06, 1.33]
[87, 226, 5.7376, 1.0, 1, 0.0712, -1.37]
[87, 227, 5.7542, 0.5, 1, 1.49, nan]
[87, 228, 5.7614, 2.0, -1, -0.76, 2.41]
